In [1]:
from sgf_to_states import sgf_to_states
from go import View

In [50]:
file_name = "../data/Mi_Yuting-Park_Junghwan.sgf"
size_board = 19
list = sgf_to_states(file_name, size_board)

In [1]:
def vis_layer(layer,num_layer):
    """Visualization of a layer
    """
    x = 0
    size = len(layer[num_layer, :, :])
    while x < size:
        string = ""
        y = 0
        while y < size:
            if layer[num_layer, y, x] == 1:
                string = string + " 1"    
            else:
                string = string + " ."
            y += 1
        print(string)
        x += 1

![title](img/Capture.JPG)

In [52]:
print(liste[0]._width)

19


In [53]:
def get_stonecolour(state):
    size = state._width
    planes = np.zeros((3, size, size))
    x = 0
    while x < size:
        y = 0
        while y < size:
            case=repr(state[x+1,y+1])
            if cell=='Black':
                planes[0, x, y] = 1 
            if cell=='White':
                planes[1, x, y] = 1 
            if cell=='Empty':
                planes[2, x, y] = 1 
            y += 1
        x += 1
    return planes

In [54]:
# stone colour
tensor_move_60 = get_stonecolour(liste[60])
print(View(liste[60]))
print(vis_layer(tensor_move_60, 0))
print(vis_layer(tensor_move_60, 1))
print(vis_layer(tensor_move_60, 2))

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . 1 1 1 . . 1 . . . . .
 . . . . . . . 1 . 1 . 1 1 . . . . . .
 . . . . 1 . . . 1 . . . . . . 1 . . .
 . . 1 . . . . . . . . . . . . . 1 1 .
 . 1 . 1 . . . . . . . . . . . 1 . 1 .
 . . . 1 . . . . . . . . . . . . . . .
 . 1

In [55]:
# ones
ones = np.ones((1, size, size))
# add ones to the first tensor
tensor_move_60 = np.concatenate([tensor_move_60, ones], axis=0)
tensor_move_60.shape

(4L, 19L, 19L)

In [85]:
def get_liberties(state):
    size = state._width
    planes = np.zeros((8, taille,taille))
    for x,y in state._legal:
        n_liberties = state.count_liberties(x,y)
        if n_liberties > 8:
            n_liberties = 8
        if not n_liberties==0:
            planes[n_liberties-1, x, y] = 1
    return planes

In [86]:
liberties = get_liberties(liste[60])
print(View(liste[60]))
print(vis_layer(liberties, 0))
print(vis_layer(liberties, 1))
print(vis_layer(liberties, 2))

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . 1 . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . 1 . .
 . . . . . . . . . . . . . . . . . . .
 . .

In [58]:
# add liberties to the first tensor
tensor_move_60 = np.concatenate([tensor_move_60, liberties], axis=0)
tensor_move_60.shape

(12L, 19L, 19L)

In [59]:
def get_history(state):
    """Doesn't exactly represents the history of the game
    but the last changement of colour of a cell
    """
    size = state._width
    planes = np.zeros((8, size, size))
    for x,y in state._legal:
        i = 1
        cell = repr(state[x, y])
        while i < 8: 
            hist = state._history[-i][0]
            celle_hist = repr(hist[x][y])
            if not (cell==hist or cell=="Empty"):
                planes[i-1, x, y] = 1
                i = 9                                    
            i += 1
        if (case != "Empty" and i==8):
            planes[7, x, y]=1
    return planes

In [61]:
history = get_history(liste[60])
print(View(liste[60]))
print(vis_layer(history, 0))
print(vis_layer(history, 1))
print(vis_layer(history, 2))

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . 1 1 1 . . 1 . . . .
 . . . . 1 . . . 1 . 1 1 1 1 1 1 . 1 .
 . . . 1 1 1 . . . 1 1 1 1 . . . 1 1 .
 . . . 1 . 1 . . . 1 . . . . . 1 1 1 1
 . . 1 . 1 1 . . . . . . . . . . 1 1 1
 . .

In [62]:
tensor_move_60 = np.concatenate([tensor_move_60, history], axis=0)
tensor_move_60.shape

(20L, 19L, 19L)

In [63]:
def get_captured(state):
    size = state._width
    planes = np.zeros((8, size, size))
    for x,y in state._legal:
        state_copy = state
        n_captured = state_copy._take_pieces(x, y)
        if n_captured > 8:
            n_captured = 8
        if n_captured != 0:
            planes[n_captured-1, x, y]=1
    return planes

In [65]:
captured = get_captured(liste[60])
print(View(liste[60]))
print(vis_layer(captured, 0))
print(vis_layer(captured, 1))
print(vis_layer(captured, 2))

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . .

In [66]:
tensor_move_60 = np.concatenate([tensor_move_60, captured], axis=0)
tensor_move_60.shape

(28L, 19L, 19L)

In [87]:
def get_liberties_after(state_after):
    planes = get_liberties(state_after)
    return planes

In [88]:
lib_after = get_liberties_after(liste[61])
print(View(liste[60]))
print(vis_layer(lib_after, 0))
print(vis_layer(lib_after, 1))
print(vis_layer(lib_after, 2))

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . 1 . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . 1 . .
 . . . . . . . . . . . . . . . . . . .
 . .

In [83]:
def get_sensibleness(state):
    size = state._width
    planes = np.ones((1, size, size))
    for x,y in state._legal:
        planes[0, x-1, y-1]=0
    return planes

In [84]:
sensibleness = get_sensibleness(liste[60])
print(View(liste[60]))
print(vis_layer(sensibleness, 0))
print(vis_layer(sensibleness, 1))
print(vis_layer(sensibleness, 2))

X . . . . . . . . . . . . . . . . . .
. . . . . . . . * * * . . * . . . . .
. . . o . . . * . * o * * o o . o . .
. . o o * . . . * o o o . . . * o . .
. . * . o . . . o . . . . . o o * * .
. * . * o . . . . . . . . . . * o * .
. . o * o . . . . . . . . . . . . . .
. * * o . . . . . . . . . . . . * . .
. * o o . . . . . . . . . . . . . . .
. . * * . . . . . + . . . . . + o . .
. o . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . o . .
. . o . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .
. . o o . . . . . + . . . . . + * . .
. . * . . . . . . . . . . * . . . . .
. * . * . o . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . .

 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 . . . 1 1 . 1 1 1 1 1
 1 1 1 . 1 1 1 . . . . . . . . 1 . 1 1
 1 1 . . . 1 1 1 . . . . 1 1 1 . . 1 1
 1 1 . 1 . 1 1 1 . 1 1 1 1 1 . . . . 1
 1 . 1 . . 1 1 1 1 1 1 1 1 1 1 . . . 1
 1 1 . . . 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 .